In [215]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os 
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average, Dense
from keras.losses import categorical_crossentropy
from keras.models import Sequential, Model, Input
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax
from keras import metrics
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from typing import Tuple, List
import numpy as np
from datetime import datetime
from matplotlib import pyplot
from keras.layers import LSTM
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [216]:
path = '/Users/joannakang/Documents/Python_study'

In [217]:
file_path = os.path.join(path,'complete_data.csv')

In [218]:
df = pd.read_csv(file_path)

In [219]:
df = df.dropna()

In [220]:
df.head()

,datetime,kWh,pressure,temperature,wind_direction,wind_speed
1,01/01/2017 00:30,2310.732265,30.19,45.0,230.0,7.0
2,01/01/2017 01:00,2289.332265,30.16,45.0,230.0,6.0
3,01/01/2017 01:30,2314.583265,30.16,45.0,240.0,7.0
4,01/01/2017 02:00,2309.308265,30.13,45.0,240.0,10.0
5,01/01/2017 02:30,2315.057265,30.13,45.0,210.0,9.0


In [221]:
df = df [['pressure','temperature','wind_direction','wind_speed']]

In [222]:
df.describe()

,pressure,temperature,wind_direction,wind_speed
count,32321.000000,32321.000000,32321.000000,32321.000000
mean,29.964390,54.884781,205.191207,10.515238
std,0.303748,10.999908,86.620754,4.663697
min,28.690000,23.000000,10.000000,1.000000
25%,29.810000,46.000000,130.000000,7.000000
50%,29.980000,54.000000,230.000000,9.000000
75%,30.160000,63.000000,260.000000,14.000000
max,30.810000,93.000000,360.000000,40.000000


In [223]:
train_df_x = df[['pressure','wind_direction','wind_speed']]

In [224]:
train_df_x.head()

,pressure,wind_direction,wind_speed
1,30.19,230.0,7.0
2,30.16,230.0,6.0
3,30.16,240.0,7.0
4,30.13,240.0,10.0
5,30.13,210.0,9.0


In [225]:
train_df_y = df['temperature']

In [226]:
train_df_y.head()

1    45.0
2    45.0
3    45.0
4    45.0
5    45.0
Name: temperature, dtype: float64

In [227]:
train_df_y.count()

32321

In [228]:
train_df_x.count()

pressure          32321
wind_direction    32321
wind_speed        32321
dtype: int64

In [229]:
x_train = train_df_x[0:20000]
y_train = train_df_y[0:20000]

In [230]:
x_val = train_df_x[20000:30000]
y_val = train_df_y[20000:30000]

In [231]:
x_test = train_df_x[30000:]
y_test = train_df_y[30000:]

In [232]:
len(x_train)

20000

In [233]:
len(y_train)

20000

In [234]:
len(x_val)

10000

In [235]:
len(y_val)

10000

In [236]:
len(x_test)

2321

In [237]:
len(y_test)

2321

In [241]:
train_df.to_csv('/Users/joannakang/Documents/Python_study/asdf.csv', header = False, index = False)

In [264]:
def deep_learning_model_1(x_size, y_size):
    t_model = Sequential() #머신러닝을 시작하겠다는 명령어
    t_model.add(Dense(100, activation="relu", input_shape=(x_size,)))
    #Dense : Fully connected? / input_shape = Input layer, 4 = Hidden layer의 노드의 갯수 #activation = Activation function의 종류를 지정
    t_model.add(BatchNormalization())
    t_model.add(Dense(100, activation="relu")) #그 다음 Hidden layer 레이어를 지정
    t_model.add(BatchNormalization())
    t_model.add(Dense(y_size)) #output layer
    print(t_model.summary())
    t_model.compile(loss='mean_squared_error',
        optimizer=Adam(lr=1e-3, decay=0.0),
        metrics=['accuracy'])
    return(t_model)

In [265]:
model = deep_learning_model_1(3,1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               400       
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 11,401
Trainable params: 11,001
Non-trainable params: 400
_________________________________________________________________
None


In [266]:
epochs = 100
batch_size = 1000

print('Epochs: ', epochs)
print('Batch_size: ', batch_size)

Epochs:  100
Batch_size:  1000


In [267]:
history = model.fit(x_train, y_train,
    batch_size=batch_size, validation_data = (x_val, y_val),
    epochs=epochs,
    shuffle=True,
    verbose=1)

Train on 20000 samples, validate on 10000 samples
Epoch 1/100
20000/20000 [==============================] - 1s 44us/step - loss: 2882.2871 - acc: 0.0000e+00 - val_loss: 3767.7071 - val_acc: 0.0000e+00
Epoch 2/100
20000/20000 [==============================] - 0s 13us/step - loss: 2846.3724 - acc: 0.0000e+00 - val_loss: 3671.4235 - val_acc: 0.0000e+00
Epoch 3/100
20000/20000 [==============================] - 0s 13us/step - loss: 2799.3918 - acc: 0.0000e+00 - val_loss: 3397.4646 - val_acc: 0.0000e+00
Epoch 4/100
20000/20000 [==============================] - 0s 13us/step - loss: 2735.2125 - acc: 0.0000e+00 - val_loss: 3127.0493 - val_acc: 0.0000e+00
Epoch 5/100
20000/20000 [==============================] - 0s 13us/step - loss: 2652.4132 - acc: 0.0000e+00 - val_loss: 3190.9103 - val_acc: 0.0000e+00
Epoch 6/100
20000/20000 [==============================] - 0s 13us/step - loss: 2550.8788 - acc: 0.0000e+00 - val_loss: 3393.0552 - val_acc: 0.0000e+00
Epoch 7/100
20000/20000 [=============

In [268]:
history_dict = history.history
history_dict.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])

In [1]:
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [273]:
train_result = model.predict(x_test)
print(train_result)

[[36.915672]
 [39.961338]
 [36.88675 ]
 ...
 [54.80735 ]
 [54.887054]
 [54.72767 ]]
